# Aquisição de dados no portal da ONS Dados Aberto

https://ons-aws-prod-opendata.s3.amazonaws.com/

https://registry.opendata.aws/ons-opendata-portal/

https://github.com/awslabs/open-data-registry/blob/main/datasets/ons-opendata-portal.yaml#L17